In [6]:
import pickle
import plotly.express as px
import pandas as pd
import numpy as np
import torch
from LogLikelihoodTorch import log_likelihood,Optimizationloop, Initialize,log_likelihood_eval,OptimizationTraj
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f)
torch.set_default_dtype(torch.float64)
X_tensor = torch.from_numpy(X).to(torch.float64)

In [2]:
# if torch.has_mps:
#     device = torch.device('mps')
# else:
#     device = torch.device('cpu')

In [3]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 7
p = 90

pi,kappa,mu = Initialize(p=p,K=K)
#%%

learning_rate = 0.4
n_iters = 3000

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)

# pi.to(device)
# mu.to(device)
# kappa.to(device)
# X_tensor.to(device)

In [14]:
LearningRates = [0.1,0.4,0.5,0.8,1,1.5,2]
LearningRates = [0.1,0.4,0.5,0.8,1,1.5,2]
Likelihoods = np.zeros((5,n_iters))

for idx,lr in enumerate(LearningRates):
    print(lr)
    pi,kappa,mu = Initialize(p=p,K=K)
    Parameters = [
        {'params':pi},
        {'params':kappa},
        {'params':mu}
    ]
    Adam = torch.optim.Adam(Parameters,lr=lr)
    Likelihoods[idx,:] = OptimizationTraj(X_tensor,[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

0.1
epoch 1; Log-Likelihood = -5529658.767288913
epoch 1001; Log-Likelihood = -6222916.469193752


KeyboardInterrupt: 

In [13]:
TrajectoryData = pd.DataFrame()
#TrajectoryData['Epoch'] = np.arange(3000)
for idx,lr in enumerate(LearningRates):
    TrajectoryData[f'lr= {lr}'] = Likelihoods[idx,:]

fig=px.line(TrajectoryData)
fig.show()

In [4]:
# Optimizing Distribution:
Pi_fit,Kappa_fit,mu_fit = Optimizationloop(X_tensor,[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -5529658.767288913
epoch 101; Log-Likelihood = -6133236.32223423
epoch 201; Log-Likelihood = -6303479.8659734735
epoch 301; Log-Likelihood = -6351940.792002776
epoch 401; Log-Likelihood = -6385889.790528606
epoch 501; Log-Likelihood = -6439478.708975163
epoch 601; Log-Likelihood = -6444570.946546835
epoch 701; Log-Likelihood = -6449694.108897326
epoch 801; Log-Likelihood = -6455465.346936359
epoch 901; Log-Likelihood = -6508370.211066372
epoch 1001; Log-Likelihood = -6522880.399422721
epoch 1101; Log-Likelihood = -6542908.651456631
epoch 1201; Log-Likelihood = -6577569.872450599
epoch 1301; Log-Likelihood = -6611099.00501027
epoch 1401; Log-Likelihood = -6640266.353915093
epoch 1501; Log-Likelihood = -6668240.531151584
epoch 1601; Log-Likelihood = -6695599.727413441
epoch 1701; Log-Likelihood = -6722648.212124977
epoch 1801; Log-Likelihood = -6749578.003818195
epoch 1901; Log-Likelihood = -6776494.803416792
epoch 2001; Log-Likelihood = -6803480.145062269
epoch

In [5]:
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [10]:
# Comparing Likelihood of pytorch vs. EM
import pickle
from WatsonDistribution import WatsonDistribution
MWD = WatsonDistribution(90)

with open('EM.pickle','rb') as f:
    Pis_EM,kappas_EM,mus_EM,states = pickle.load(f)

#log_Likelihood_EM = sum(np.log([sum([ Pis_EM[j]* MWD.pdf(x,mus_EM[:,j],kappas_EM[j]) for j in range(K)]) for x in X.T]))
log_Likelihood_EM =  log_likelihood_eval(X_tensor,torch.from_numpy(Pis_EM),torch.from_numpy(kappas_EM),torch.from_numpy(mus_EM),p=90,K=7)
log_Likelihood_torch = log_likelihood_eval(X_tensor,pi_est,kappa_est,mu_est,p=90,K=7)

print("Likelihood Camparison:")
print(f"EM-Algortihem LogLikelihood: L = {log_Likelihood_EM}")
print(f"Torch Optimization of LogLikelihood: L = {log_Likelihood_torch}")

# Likelihood Camparison:
# EM-Algortihem LogLikelihood: L = 5642240.768217143

Likelihood Camparison:
EM-Algortihem LogLikelihood: L = 6469526.793269191
Torch Optimization of LogLikelihood: L = 6963787.030903634


In [11]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {Pis_EM[model]} , kappa = {kappas_EM[model]}")
    fig = px.bar(y=mus_EM[:,model],x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.15679143264517995 , kappa = 72.80264462628915


Model 1: Pi = 0.1537378367678051 , kappa = 62.089876938688164


Model 2: Pi = 0.13077628873636674 , kappa = 76.17904116835214


Model 3: Pi = 0.1411151156992721 , kappa = 70.94346750356566


Model 4: Pi = 0.1647789740863885 , kappa = 73.4130260903616


Model 5: Pi = 0.09973201322787797 , kappa = 86.32534755132596


Model 6: Pi = 0.15306833883710966 , kappa = 63.53188179780241


In [12]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {pi_est[model]} , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.05546220086936685 , kappa = 720.2431077049829


Model 1: Pi = 0.1905627019984416 , kappa = 73.74833965841405


Model 2: Pi = 0.20610212194958505 , kappa = 61.22801700846581


Model 3: Pi = 0.16848052008841807 , kappa = 67.88046091797173


Model 4: Pi = 0.1502364358109449 , kappa = 93.79499993057495


Model 5: Pi = 0.17747107915879295 , kappa = 71.47942272857838


Model 6: Pi = 0.051684940124450525 , kappa = 804.4309444843584
